In [1]:
import requests
import datetime
from urllib.parse import urlencode
import base64
from collections import defaultdict
import pandas as pd

from IPython.display import display
from IPython.display import Image, HTML
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_colwidth', -1)
pd.options.display.max_columns = None

C:\Users\rhbar\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  # This is added back by InteractiveShellApp.init_path()


In [2]:
class SpotifyAPI(object):
    user_id = None
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, user_id, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        self.user_id = user_id

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()

    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {"Authorization": f"Basic {client_creds_b64}"}

    def get_token_data(self):
        return {"grant_type": "client_credentials"} 

    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True

    def headers(self):
        if self.perform_auth():
            pass
        else:
            self.perform_auth()
        access_token = self.access_token
        return {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}',
        }

    def get_user_playlists(self):
        headers = self.headers()
        user_id = self.user_id
        all_user_playlists = f"https://api.spotify.com/v1/users/{user_id}/playlists"
        playlist_meta = defaultdict(list)
        scopes = urlencode({"scope": "playlist-read-private playlist-read-collaborative"})
        playlists_URL = f'{all_user_playlists}?{scopes}'
        r = requests.get(playlists_URL, headers=headers)
        data = r.json()
        playlists = data['total']
        for playlist in range(playlists):
            playlist_meta[playlist] = {
            "name":data['items'][playlist]["name"],
            "id": data['items'][playlist]["id"],
            "total_tracks": data['items'][playlist]["tracks"]["total"],
            "owner": data['items'][playlist]['owner']["display_name"]
            }
        df_playlist = pd.DataFrame.from_dict(playlist_meta, orient='index')
        return df_playlist

    def get_playlist_tracks(self):
        playlists = self.get_user_playlists()
        headers = self.headers()
        print(playlists)
        print("\n------------------------------")
        playlist_number = int(input("Enter your playlist: "))
        print(playlists.iloc[[playlist_number]],"\n------------------------------\n\n\n")
        playlist_id = playlists.iloc[playlist_number][1]
        r = requests.get(f'https://api.spotify.com/v1/playlists/{playlist_id}', headers=headers)
        data = r.json()
        songs = data['tracks']["total"]
        songs_dict = defaultdict(list)
        for song in range(songs):
            artist_names = ""
            artists = data['tracks']['items'][song]['track']["artists"]
            for artist in range(len(artists)):
                artist_names += (data['tracks']['items'][song]['track']["artists"][artist]['name'] + ' | ')
                artist_names += (data['tracks']['items'][song]['track']["artists"][artist]['id'] + '; ')
            songs_dict[song] = {
            'track': data['tracks']['items'][song]['track']['name'],
            'id': data['tracks']['items'][song]['track']['id'],
            'artist': artist_names,
            'album_name': data['tracks']['items'][song]['track']["album"]['name'],
            'album_id': data['tracks']['items'][song]['track']["album"]['id'],
            'album_art': data['tracks']['items'][song]['track']["album"]['images'][0]['url']}
            del artist_names
        songs_df = pd.DataFrame.from_dict(songs_dict, orient='index')

#         songs_df.to_csv(r'sample.csv', index=False, encoding='utf-8', sep=",")
        return(songs_df)

In [3]:
client_id = "4a5e59ebfee347ef9ebf4bfd928ff106"
client_secret = "45fd6d9880664c0abe6657ef3bedba5d"
user_id = "rj9f4jebczrub9ps4gutenlyn"

In [4]:
spotify = SpotifyAPI(client_id, client_secret, user_id)

playlists = spotify.get_playlist_tracks()

            name                      id  total_tracks              owner
0  Hollywood      7ufevxWlRz5DbRS7BnQTsE  4             Sanju Raghuwanshi
1  sanju          5mIhVz7zh0D5RrmLF0HlGk  1             Sanju Raghuwanshi
2  My Favourites  5Ub6PoeVMhsThbzuBLtSXA  192           Sanju Raghuwanshi

------------------------------
Enter your playlist: 0
        name                      id  total_tracks              owner
0  Hollywood  7ufevxWlRz5DbRS7BnQTsE  4             Sanju Raghuwanshi 
------------------------------





In [5]:
playlists

,track,id,artist,album_name,album_id,album_art
0,Lean On (with Emiway Bantai),61eDLrQvquaTtZRnGFz9Kh,Celina Sharma | 42daDEQTdYaqtHG5sH5HMD; Emiway Bantai | 008PpLcKUtVXle6JSwkq3I;,Lean On (with Emiway Bantai),6h53Qu0rJsJuOrVnEXpxW7,https://i.scdn.co/image/ab67616d0000b273754a7b0af59c3a88c9d5173e
1,ROXANNE,696DnlkuDOXcMAnKlTgXXK,Arizona Zervas | 0vRvGUQVUjytro0xpb26bs;,ROXANNE,6HJDrXs0hpebaRFKA1sF90,https://i.scdn.co/image/ab67616d0000b273069a93617a916760ab88ffea
2,"My Heart Will Go On - Love Theme from ""Titanic""",33LC84JgLvK2KuW43MfaNq,Céline Dion | 4S9EykWXhStSc15wEx8QFK;,Let's Talk About Love,1tfS7Fo1UtAxQSf256fnYs,https://i.scdn.co/image/ab67616d0000b273745adc3f697ea1ec79d66901
3,It's Wednesday,7oJzahtQz2RfGnheA1HgOy,Spotify | 5UUG83KSlqPhrBssrducWV;,Daily Sports,6IkndfRrafR77MUJKHgIri,https://i.scdn.co/image/ab67616d0000b27392a7f50c9e301b11a2556719
